## PREPROCESSING

Ricette definite in un formato semistrutturato, raggruppamento degli step e ingredienti in un unica stringa, funzioni di utilità

### Funzioni util e import

In [1]:
from pathlib import Path
import pandas as pd
from IPython.display import display
from ast import literal_eval
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
from nltk.sem import relextract
import spacy
from spacy import displacy
from nltk.corpus import conll2000
from spacy.symbols import X, NUM, VERB, NOUN
from nltk.corpus import wordnet as wn

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('conll2000')


!python -m spacy download en_core_web_md

NLP = spacy.load('en_core_web_md')


def string_recipe(i):
    return dataset.iloc[i]['title'] + "\n\n" + dataset.iloc[i]['ingredients'] + "\n\n" + dataset.iloc[i]['step'] 

[nltk_data] Downloading package stopwords to /home/navis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/navis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/navis/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/navis/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/navis/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package conll2000 to /home/navis/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!


     |████████████████████████████████| 45.7 MB 24.0 MB/s eta 0:00:01   |████▋                           | 6.6 MB 2.5 MB/s eta 0:00:16     |████████▎                       | 11.8 MB 2.5 MB/s eta 0:00:14     |██████████████████▉             | 27.0 MB 3.1 MB/s eta 0:00:06     |████████████████████▍           | 29.2 MB 3.1 MB/s eta 0:00:06     |███████████████████████████████▎| 44.6 MB 3.1 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


### Caricamento del dataset

In [2]:
dataset = pd.read_csv(
    Path("../data/test_dataset.csv").resolve(), 
    index_col=[0], 
    names=["index", "title","ingredients","step"], 
    usecols=[0,1,2,3]
    )

for index in range(len(dataset)):
    dataset.iloc[index]['ingredients'] = ".\n".join(literal_eval(dataset.iloc[index]['ingredients']))
    dataset.iloc[index]['step'] = " ".join(literal_eval(dataset.iloc[index]['step']))

display(dataset.head())
print(string_recipe(10))


,title,ingredients,step
index,,,
1,Jewell Ball'S Chicken,"1 small jar chipped beef, cut up.\n4 boned chi...",Place chipped beef on bottom of baking dish. P...
2,Creamy Corn,2 (16 oz.) pkg. frozen corn.\n1 (8 oz.) pkg. c...,"In a slow cooker, combine all ingredients. Cov..."
3,Chicken Funny,1 large whole chicken.\n2 (10 1/2 oz.) cans ch...,Boil and debone chicken. Put bite size pieces ...
4,Reeses Cups(Candy),1 c. peanut butter.\n3/4 c. graham cracker cru...,Combine first four ingredients and press in 13...
5,Cheeseburger Potato Soup,6 baking potatoes.\n1 lb. of extra lean ground...,Wash potatoes; prick several times with a fork...


Buckeye Candy

1 box powdered sugar.
8 oz. soft butter.
1 (8 oz.) peanut butter.
paraffin.
12 oz. chocolate chips

Mix sugar, butter and peanut butter. Roll into balls and place on cookie sheet. Set in freezer for at least 30 minutes. Melt chocolate chips and paraffin in double boiler. Using a toothpick, dip balls 3/4 of way into chocolate chip and paraffin mixture to make them look like buckeyes.


### Estrazione delle abbreviazioni

Fase iniziale di ritrovamento del set di abbreviazioni

In [3]:
abbrv_dataset = pd.read_csv(
    Path("../data/test_dataset.csv").resolve(), 
    index_col=[0], 
    names=["index", "title","ingredients","step"], 
    usecols=[0,1,2,3]
    )

abbrv = set()
for index in range(len(dataset)):
    abbrv_dataset.iloc[index]['ingredients'] = " ".join(literal_eval(abbrv_dataset.iloc[index]['ingredients']))
    for element in re.findall(r"[A-Za-z]*\.", abbrv_dataset.iloc[index]['ingredients']):
        abbrv.add(element)
    
print(abbrv)

{'c.', 'tsp.', 'tbsp.', 'gal.', 'No.', 'qt.', 'oz.', 'pkg.', 'Tbsp.', 'pt.', 'sq.', 'lb.'}


Rimozione delle abbreviazioni in quanto possono essere dannose per il processo di tokenizzazione. Es pkg. ---> package

In [4]:
def expand_abbreviations(ingredients_string):
    __ABBREVIATIONS__ = {
        'pkg.'  :   'package',
        'tsb.'  :   'tablespoon',
        'no.'   :   'number',
        'pt.'   :   'pint',
        'no.'   :   'number',
        'gal.'  :   'gallon',
        'tbsp.' :   'tablespoon',
        'sq.'   :   'square',
        'oz.'   :   'ounce',
        'lb.'   :   'pound',
        'qt.'   :   'quart',
        'c.'    :   'cup',
        'tsp.'  :   'teaspoon'
    }
    for item, value in __ABBREVIATIONS__.items():
        ingredients_string = ingredients_string.lower().replace(item, value)
    return ingredients_string


for index in range(len(dataset)):
    dataset.iloc[index]['ingredients'] = expand_abbreviations(dataset.iloc[index]['ingredients'])

### Sentence splitting
Il contenuto delle colonne 'ingredients' e 'step' verrà suddiviso in frasi. In precedenza i periodi contenuti nelle singole celle sono stati formattati in modo tale da renderli riconoscibili e facilmente suddivisibili in frasi ben separate.

In [5]:
for index in range(len(dataset)):
     dataset.iloc[index]['ingredients'] = (sent_tokenize(dataset.iloc[index]['ingredients']))
print(dataset.iloc[10]['ingredients'])

['1 box powdered sugar.', '8 ounce soft butter.', '1 (8 ounce) peanut butter.', 'paraffin.', '12 ounce chocolate chips']


In [6]:
for index in range(len(dataset)):
     dataset.iloc[index]['step'] = (sent_tokenize(dataset.iloc[index]['step']))
print(dataset.iloc[10]['step'])

['Mix sugar, butter and peanut butter.', 'Roll into balls and place on cookie sheet.', 'Set in freezer for at least 30 minutes.', 'Melt chocolate chips and paraffin in double boiler.', 'Using a toothpick, dip balls 3/4 of way into chocolate chip and paraffin mixture to make them look like buckeyes.']


### Rimozione quantità doppie

In [7]:
dr_reg = r'\d*\s*\(.*\)'

for index in range(len(dataset)):
    for value in range(len(dataset.iloc[index]['ingredients'])):
        elements = re.findall(dr_reg, dataset.iloc[index]['ingredients'][value])
        for e in elements:
            new_string = dataset.iloc[index]['ingredients'][value].replace(e,e[e.find("(")+1: e.find(")")].strip())
            dataset.iloc[index]['ingredients'][value] = new_string

## ANALISI

### Stop words removal
Nella colonna 'step' troviamo una serie di passaggi da compiere per creare la ricetta. Questi passaggi sono scritti in linguaggio naturale e possono essere semplificati rimuovendo delle parole dette stop words.

In [ ]:
# OPTINAL -> Risultati in swr_dataset"

stop_words = set(stopwords.words('english'))
swr_dataset = dataset.copy(deep=True)

tk = lambda x,st: ' '.join([w for w in x if w not in st])
for index in range(len(dataset)):
    swr_dataset.iloc[index]['step'] = [tk(word_tokenize(sent), stop_words) for sent in dataset.iloc[index]['step']]

### Stemming e Lemming
Questi due processi potrebbero portare valore all'analisi del dominio. Il codice per entrambi viene proposto qui.

In [ ]:
# OPTINAL -> Risultati in lem_dataset"

lemmatizer = WordNetLemmatizer()
lem_dataset = swr_dataset.copy(deep=True)

for index in range(len(swr_dataset)):
    lem_dataset.iloc[index]['step'] = stem_sent = [' '.join([lemmatizer.lemmatize(w) for w in word_tokenize(sent)]) for sent in swr_dataset.iloc[index]['step']]

In [ ]:
# OPTINAL -> Risultati in stm_dataset"

stemmer = PorterStemmer()
stm_dataset = lem_dataset.copy(deep=True)

for index in range(len(lem_dataset)):
    stm_dataset.iloc[index]['step'] = [' '.join([stemmer.stem(w) for w in word_tokenize(sent)]) for sent in lem_dataset.iloc[index]['step']]

In [ ]:
print("Frasi originali\n")
print(dataset.iloc[10]['step'])
print("\nStop word removal\n")
print(swr_dataset.iloc[10]['step'])
print("\nStop word e lemming\n")
print(lem_dataset.iloc[10]['step'])
print("\nStop word lemming e stemming\n")
print(stm_dataset.iloc[10]['step'])

### Entityt Extraction, Relation Extraction e POS tagging con SpaCy

In [9]:
spacy_dataset = dataset.copy(deep=True)
       
for index in range(len(dataset)):
    spacy_dataset.iloc[index]['ingredients'] = [NLP(element) for element in dataset.iloc[index]['ingredients']]
    spacy_dataset.iloc[index]['step'] = [NLP(element) for element in dataset.iloc[index]['step']]

In [10]:
def __pprintsp__(dataset, index, column, value, extend=False):
    displacy.render(dataset.iloc[index][column][value], style='dep')
    print(dataset.iloc[index][column][value])
    displacy.render(dataset.iloc[index][column][value], style='ent')
    if extend:
        for token in dataset.iloc[index][column][value]:
            print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)

def pprint_spacyd_all(dataset, index, column, extend=False):
    for v in range(len(dataset.iloc[index][column])):
        __pprintsp__(dataset, index, column, v, extend)
            
def pprint_spacyd(dataset, index, column, value, extend=False):
    __pprintsp__(dataset, index, column, value, extend)

### Analisi degli ingredienti

In [54]:
class Recipe:
    def __init__(self, title, index, ingredients_str, steps_str):
        self.title = title
        self.ingredients = []
        self.ing_str = ingredients_str
        self.stp_str = steps_str
        self.steps = []
        self.idx = index
    
    def add_ing(self, ing):
        self.ingredients.append(ing)
        
    def __str__(self):
        _ing_s = '\n'.join([ing.short_srt() for ing in self.ingredients])
        return f"Title: {self.title}\n{_ing_s}"

class Ingredient:
    def __init__(self, full_text):
        self.name=""
        self.ing_cat_id = None
        self.size=""
        self.quantity=""
        self.size=""
        self.adj=[]
        self.original=full_text
    def __str__(self):
        return f"Name: {self.name}\nAjdectives: {self.adj}\nQuantity: {self.quantity} {self.size}\nOriginal: {self.original}"

    def short_srt(self):
        return f"{self.name} ({self.adj}) {self.quantity} {self.size}"
    
    def set_id(self, identifier):
        self.ing_cat_id = identifier

In [58]:

def is_food(word):
    syns = wn.synsets(str(word), pos = wn.NOUN)
    for syn in syns:
        if 'food' in syn.lexname():
            return True
    return False


#Root generation

__SIZES__ = [
       'package',
       'tablespoon',
       'number',
       'pint',
       'number',
       'gallon',
       'tablespoon',
       'square',
       'ounce',
       'pound',
       'quart',
       'cup',
       'teaspoon',
       'can'
]


all_recipes = []
    
for index in range(len(spacy_dataset)):
    
    recipe = Recipe(
        spacy_dataset.iloc[index]['title'], 
        index, 
        " ".join(dataset.iloc[index]['ingredients']),
        " ".join(dataset.iloc[index]['step'])       
    )
    
    for element in spacy_dataset.iloc[index]['ingredients']:

        ingredient = element

        IGN = Ingredient(ingredient.text)
        ROOT_NODE = [token for token in ingredient if token.dep_ == 'ROOT'][0]
        

        if (ROOT_NODE.pos == VERB):
                IGN.adj.append(ROOT_NODE.text)
        else:
                IGN.name = ROOT_NODE.text
                
        stack = [element for element in ROOT_NODE.children]

        while len(stack)!=0:
            CURRENT_NODE = stack.pop()

            if (CURRENT_NODE.pos == X) or (CURRENT_NODE.pos == NUM):
                    IGN.quantity += " " + CURRENT_NODE.text
            else:
                if CURRENT_NODE.text.lower() in  __SIZES__:
                    IGN.size += " " + CURRENT_NODE.text

                if ((CURRENT_NODE.dep_ == 'compound') or (CURRENT_NODE.dep_ == 'dobj') or (CURRENT_NODE.dep_ == 'pobj')) and is_food(CURRENT_NODE.text) and (CURRENT_NODE.text.lower() not in  __SIZES__):
                    IGN.name = CURRENT_NODE.text + " " + IGN.name

                if (CURRENT_NODE.dep_ == 'amod') or (CURRENT_NODE.dep_ == 'appos'):
                    if is_food(CURRENT_NODE.text):
                        IGN.name = CURRENT_NODE.text + " " + IGN.name
                    else:
                        IGN.adj.append(CURRENT_NODE.text)         

            stack += [element for element in CURRENT_NODE.children]
            
        recipe.add_ing(IGN)
    all_recipes.append(recipe)
        


### Analisi degli step

In [16]:
PREPS = ['with', 'in', 'on', 'of']

#Questa funzione potrebbe essere migliorata in mille modi, per ora me la faccio bastare
def find_instrument(child):
    if child.dep_ == 'prep' and child.text in PREPS:
        for subchild in child.children:
            if subchild.dep_ == 'pobj' and not is_food(subchild.text):
                for subsubchild in subchild.children:
                    if subsubchild.dep_ == 'prep' and subsubchild.text in PREPS:
                        if subsubchild.dep_ == 'pobj' and not is_food(subchild.text):
                            ins = subsubchild.text
                            for conn in subsubchild.children:
                                if conn.dep_ == 'compound':
                                    ins = conn.text + ' ' + ins
                            return ins
                    else:
                        ins = subchild.text
                        for conn in subchild.children:
                            if conn.dep_ == 'compound':
                                ins = conn.text + ' ' + ins
                        return ins
    else:
        return False

#Questa funzione sono riuscito a farla leggermente meglio e funziona discretamente bene, 
# potrebbe essere fatta qualche miglioria ma tutto sommato fa il suo dovere
def find_ingredient(child):
    ing_list = []
    ing = child.text
    for subchild in child.children:
        if subchild.dep_ == 'amod':
            ing = subchild.text + ' ' + ing
    ing_list.append(ing)
    for subchild in child.children:
        if subchild.dep_ == 'conj':
            ing_list.append(find_ingredient(subchild).pop())
    return ing_list
                
actions = set()
instruments = set()
ingredients_conn = set()

for index in range(len(spacy_dataset)):
    for doc in spacy_dataset.iloc[index]['step']:
        for token in doc:
            #Cerco i verbi
            if token.pos_ == 'VERB':
                #AZIONE TROVATA (per ora sono tutti i verbi, effettuare un controllo ulteriore?)
                act = token.text
                actions.add(act)

                for child in token.children:
                    #Tra tutti i child del verbo mi cerco lo strumento tramite la funzione find_instrument
                    if find_instrument(child):
                        ins = find_instrument(child)
                        instruments.add(ins)
                    #Se un child ha dipendenza dobj è un ingrediente e chiamo la funzione find_ingredient che mi ritorna una lista di ingredienti, aggiungo singolarmente ognuno di essi al set
                    if child.dep_ == 'dobj':
                        ing_list = find_ingredient(child)
                        for ing in ing_list:
                            ingredients_conn.add(ing)
print(actions)
print(instruments)
print(ingredients_conn)

## GENERAZIONE DATABASE

In [56]:
macro_ingredients = {}
ingredients_idx = {}

for element in all_recipes:
    for ingredient in element.ingredients:
        if ingredient.name in macro_ingredients:
            macro_ingredients[ingredient.name] += 1
        else:
            macro_ingredients[ingredient.name] = 1
            
for index, key in enumerate(macro_ingredients):
    ingredients_idx[key] = index
    print(f"{key:>50s} {macro_ingredients[key]:<3d}")

                                             beef  3  
                                   chicken breasts 2  
                               mushroom soup cream 4  
                                        sour cream 9  
                                              corn 2  
                                      cream cheese 8  
                                            butter 12 
                                     garlic powder 1  
                                              salt 22 
                                            pepper 10 
                                           chicken 2  
                                chicken gravy cans 1  
                                          stuffing 1  
                                            cheese 2  
                                     peanut butter 4  
                             graham cracker crumbs 3  
                                           butter  5  
                                         sugar cup 1  
          

In [60]:
print(all_recipes[0].ing_str)
print(all_recipes[0].stp_str)

1 small jar chipped beef, cut up. 4 boned chicken breasts. 1 can cream of mushroom soup. 1 carton sour cream
Place chipped beef on bottom of baking dish. Place chicken on top of beef. Mix soup and cream together; pour over chicken. Bake, uncovered, at 275° for 3 hours.


In [63]:
ING_DB_CREATE = """
USE recipe_analysis;
DROP TABLE IF EXISTS ingredients;
CREATE TABLE ingredients(
id INT PRIMARY KEY,
name VARCHAR(100) NOT NULL
);
"""

print(ING_DB_CREATE)


with open(Path("../database/create_ingredients.sql").resolve(), "w") as ddl_file:
    for ing in ingredients_idx:
        ING_DB_CREATE = ING_DB_CREATE + f"INSERT INTO ingredients VALUES ({ingredients_idx[ing]},\"{ing}\");\n"
    ddl_file.write(ING_DB_CREATE)


USE recipe_analysis;
DROP TABLE IF EXISTS ingredients;
CREATE TABLE ingredients(
id INT PRIMARY KEY,
name VARCHAR(100) NOT NULL
);



In [51]:
display(dataset)

,title,ingredients,step
index,,,
1,Jewell Ball'S Chicken,"[1 small jar chipped beef, cut up., 4 boned ch...","[Place chipped beef on bottom of baking dish.,..."
2,Creamy Corn,"[16 ounce package frozen corn., 8 ounce packag...","[In a slow cooker, combine all ingredients., C..."
3,Chicken Funny,"[1 large whole chicken., 10 1/2 ounce cans chi...","[Boil and debone chicken., Put bite size piece..."
4,Reeses Cups(Candy),"[1 cup peanut butter., 3/4 cup graham cracker ...",[Combine first four ingredients and press in 1...
5,Cheeseburger Potato Soup,"[6 baking potatoes., 1 pound of extra lean gro...",[Wash potatoes; prick several times with a for...
...,...,...,...
94,Beef And Spanish Rice Casserole,"[1 can spanish rice., 1 can corned beef., 1 di...","[Spray a 1-quart casserole dish with Pam., Lay..."
95,Baked Beans,"[1 pound cans pork and beans., 1/2 cup bell pe...",[Cook onions and bell pepper in oil until onio...
96,Sweet-N-Sour Chicken,"[2 cup diced cooked chicken., 2 tablespoon sho...","[Saute onion in 2 tablespoons shortening., Add..."


In [80]:
REC_DB_CREATE = """
USE recipe_analysis;
DROP TABLE IF EXISTS recipes;
CREATE TABLE recipes(
id INT PRIMARY KEY,
titolo VARCHAR(100) NOT NULL,
preparazione_nstr VARCHAR(500),
ingredienti_nstr VARCHAR(500)
);
"""

print(REC_DB_CREATE)



with open(Path("../database/create_recipes.sql").resolve(), "w") as ddl_file:
    for rec in all_recipes:
        
        corrected_ing = rec.ing_str.replace("\'","\'\'")
        corrected_stp = rec.stp_str.replace("\'","\'\'")
        corrected_tit = rec.title.replace("\'","\'\'")
        
        REC_DB_CREATE = REC_DB_CREATE + f"INSERT INTO recipes (id, titolo, ingredienti_nstr, preparazione_nstr) VALUES ({rec.idx},\'{corrected_tit}\',\'{corrected_ing}\',\'{corrected_stp}\');\n"
    ddl_file.write(REC_DB_CREATE)


USE recipe_analysis;
DROP TABLE IF EXISTS recipes;
CREATE TABLE recipes(
id INT PRIMARY KEY,
titolo VARCHAR(100) NOT NULL,
preparazione_nstr VARCHAR(500),
ingredienti_nstr VARCHAR(500)
);



In [84]:
CON_DB_CREATE = """
USE recipe_analysis;
DROP TABLE IF EXISTS contains;
CREATE TABLE contains(
id INT NOT NULL AUTO_INCREMENT,
misura VARCHAR(50),
quantita VARCHAR(50),
proprieta VARCHAR(100),
recipeID int NOT NULL,
ingredientID int NOT NULL,
FOREIGN KEY (recipeID) REFERENCES recipes(id),
FOREIGN KEY (ingredientID) REFERENCES ingredients(id)
);
"""

print(CON_DB_CREATE)


with open(Path("../database/create_contains.sql").resolve(), "w") as ddl_file:
    for r in all_recipes:
        for i in r.ingredients:
            adj = ', '.join(i.adj)
            data = f"(\'{i.size}\',\'{i.quantity}\',\'{adj}\',{r.idx}, {ingredients_idx[i.name]});\n"

            print(data)
        
        
        
      


USE recipe_analysis;
DROP TABLE IF EXISTS contains;
CREATE TABLE contains(
id INT NOT NULL AUTO_INCREMENT,
misura VARCHAR(50),
quantita VARCHAR(50),
proprieta VARCHAR(100),
recipeID int NOT NULL,
ingredientID int NOT NULL,
FOREIGN KEY (recipeID) REFERENCES recipes(id),
FOREIGN KEY (ingredientID) REFERENCES ingredients(id)
);

('',' 1','cut, chipped, small',0, 0);

('',' 4','boned',0, 1);

(' can',' 1','',0, 2);

('',' 1','',0, 3);

(' package ounce',' 16','frozen',1, 4);

(' package ounce',' 8','cubed',1, 5);

(' cup','','cubed',1, 6);

(' teaspoon',' 1/2','',1, 7);

(' teaspoon',' 1/2','',1, 8);

(' teaspoon',' 1/4','',1, 9);

('',' 1','whole, large',2, 10);

(' ounce',' 1/2 10','',2, 11);

(' can ounce',' 1/2 10','',2, 2);

(' ounce',' 6','',2, 12);

(' ounce',' 4','shredded',2, 13);

(' cup',' 1','',3, 14);

(' cup',' 3/4','',3, 15);

(' cup',' 1','melted',3, 16);

('',' 1/2 1','powdered',3, 17);

(' package',' 1','large',3, 18);

('',' 6','',4, 19);

('',' 1','lean, extra',4, 20);